<a href="https://colab.research.google.com/github/RMoulla/DL_Pytorch/blob/main/TP_GNN_GAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP : Réseaux de neurones graphiques à base d'Attention.

Dans ce TP, nous nous proposons d'étudier le dataset Cora. Celui-ci est composé de 2708 papiers scientifiques répartis en sept classes. L'objectif de ce TP est de construire un modèle permettant de prédire la classe à laquelle appartient un noeud (papier scientifique).



In [ ]:
import os
import torch
import networkx as nx
import pandas as pd
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.6.0+cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
import torch_geometric.transforms as T

import matplotlib.pyplot as plt

## Analyse du dataset Cora

On se propose ici d'explorer le dataset Cora. Il est fortement recommandé de compléter cette analyse exploratoire (visualiser le graphe, etc.).

In [ ]:

dataset = Planetoid(root='~/somewhere/Cora', name='Cora')

data = dataset[0]
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')

Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Contains isolated nodes: False
Contains self-loops: False


Processing...
Done!
/tmp/ipython-input-3-383367433.py:15: UserWarning: 'contains_isolated_nodes' is deprecated, use 'has_isolated_nodes' instead
  print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
/tmp/ipython-input-3-383367433.py:16: UserWarning: 'contains_self_loops' is deprecated, use 'has_self_loops' instead
  print(f'Contains self-loops: {data.contains_self_loops()}')


## Question 1 : construire un modèle à base de GAT qui permet de prédire la classe à laquelle appartient un noeud.
---



In [5]:
import torch.nn as nn
import torch.nn.functional as F

class GATNet(nn.Module):
    def __init__(self, num_features, num_classes):
        super(GATNet, self).__init__()
        self.conv1 = GATConv(num_features, 8, heads=8, dropout=0.6)
        self.conv2 = GATConv(8 * 8, num_classes, heads=1, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

# Créer le modèle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GATNet(dataset.num_node_features, dataset.num_classes).to(device)
data = dataset[0].to(device)

# Fonction de perte et optimiseur
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

# Fonction d'entraînement
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

# Fonction de test
def test():
    model.eval()
    logits, accs = model(data), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

# Boucle d'entraînement
for epoch in range(200):
    loss = train()
    train_acc, val_acc, test_acc = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')


Epoch: 000, Loss: 1.9993, Train: 0.5143, Val: 0.3600, Test: 0.3420
Epoch: 001, Loss: 1.9196, Train: 0.7786, Val: 0.5300, Test: 0.5020
Epoch: 002, Loss: 1.8545, Train: 0.8357, Val: 0.5960, Test: 0.5740
Epoch: 003, Loss: 1.7803, Train: 0.8714, Val: 0.6480, Test: 0.6200
Epoch: 004, Loss: 1.6487, Train: 0.8786, Val: 0.6860, Test: 0.6670
Epoch: 005, Loss: 1.5234, Train: 0.9000, Val: 0.7080, Test: 0.6960
Epoch: 006, Loss: 1.4656, Train: 0.9143, Val: 0.7260, Test: 0.7210
Epoch: 007, Loss: 1.3824, Train: 0.9357, Val: 0.7400, Test: 0.7300
Epoch: 008, Loss: 1.3965, Train: 0.9500, Val: 0.7360, Test: 0.7290
Epoch: 009, Loss: 1.4369, Train: 0.9500, Val: 0.7300, Test: 0.7300
Epoch: 010, Loss: 1.3526, Train: 0.9500, Val: 0.7320, Test: 0.7410
Epoch: 011, Loss: 1.2129, Train: 0.9500, Val: 0.7260, Test: 0.7460
Epoch: 012, Loss: 1.2498, Train: 0.9571, Val: 0.7340, Test: 0.7460
Epoch: 013, Loss: 1.1285, Train: 0.9571, Val: 0.7380, Test: 0.7470
Epoch: 014, Loss: 1.1462, Train: 0.9643, Val: 0.7400, Test: 0.